In [48]:
# Data manipulation
import numpy as np
import pandas as pd
import random

# Data visualisation
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.utils
import torchvision.datasets as dset

from torch import optim
from torch.utils.data import DataLoader,Dataset
from torchvision.models import *
from torchvision.datasets import ImageFolder
from torch.autograd import Variable

from pathlib import Path
import sys
import os
from glob import glob
from PIL import Image

In [49]:
np.random.seed(42)
NUMBER_EPOCHS=100
IMG_SIZE=100

def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

In [50]:

val_famillies = "F09"
all_images = glob(os.getcwd() +"/input/train/*/*/*.jpg")
all_images = [path.replace("\\", "/") for path in all_images]
ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]
relationships = pd.read_csv(os.getcwd() + "/input/train_relationships.csv")
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if val_famillies not in x[0]]
val = [x for x in relationships if val_famillies in x[0]]

print("Total train pairs:", len(train))    
print("Total val pairs:", len(val))    

Total train pairs: 3066
Total val pairs: 296


In [51]:
class trainingDataset(Dataset):
    
    def __init__(self,imageFolderDataset, relationships, transform=None):
        self.imageFolderDataset = imageFolderDataset    
        self.relationships = relationships #choose either train or val dataset
        self.transform = transform
        
    def __getitem__(self,index):
        img0_info = self.relationships[index][0]
        img0_path = glob(os.getcwd() +"/input/train/"+img0_info+"/*.jpg")
        img0_path = [path.replace("\\", "/") for path in img0_path]
        img0_path = random.choice(img0_path)
        
        cand_relationships = [x for x in self.relationships if x[0]==img0_info or x[1]==img0_info]
        if cand_relationships==[]:
            should_get_same_class = 0
        else:
            should_get_same_class = random.randint(0,1) 

        if should_get_same_class==1:
            img1_info = random.choice(cand_relationships)
            if img1_info[0]!=img0_info:
                img1_info=img1_info[0]
            else:
                img1_info=img1_info[1]
            img1_path = glob(os.getcwd() + "/input/train/"+img1_info+"/*.jpg")#randomly choose a img of this person
            img1_path = random.choice(img1_path)
        else:#0 means non-related
            randChoose = True#in case the chosen person is related to first person
            while randChoose:
                img1_path = random.choice(self.imageFolderDataset.imgs)[0]
                img1_info = img1_path.split("/")[-3] + "/" + img1_path.split("/")[-2]
                randChoose = False
                for x in cand_relationships:#if so, randomly choose another person
                    if x[0]==img1_info or x[1]==img1_info:
                        randChoose = True
                        break
                    
        img0 = Image.open(img0_path)
        img1 = Image.open(img1_path)
        
        if self.transform is not None:#
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , should_get_same_class
    
    def __len__(self):
        return len(self.relationships)

In [52]:
folder_dataset = dset.ImageFolder(root=os.getcwd() +"/input/train")

trainset = trainingDataset(imageFolderDataset=folder_dataset,
                                        relationships=train,
                                        transform=transforms.Compose([transforms.Resize((IMG_SIZE,IMG_SIZE)),
                                                                      transforms.ToTensor()
                                                                      ]))
trainloader = DataLoader(trainset,
                        shuffle=True,#whether randomly shuffle data in each epoch, but cannot let data in one batch in order.
                        batch_size=BATCH_SIZE)
valset = trainingDataset(imageFolderDataset=folder_dataset,
                                        relationships=val,
                                        transform=transforms.Compose([transforms.Resize((IMG_SIZE,IMG_SIZE)),
                                                                      transforms.ToTensor()
                                                                      ]))
valloader = DataLoader(valset,
                        shuffle=True,
                        batch_size=BATCH_SIZE)

vis_dataloader = DataLoader(trainset,
                        shuffle=True,
                        batch_size=8)
dataiter = iter(vis_dataloader)


example_batch = next(dataiter)
concatenated = torch.cat((example_batch[0],example_batch[1]),0)
imshow(torchvision.utils.make_grid(concatenated))
print(example_batch[2].numpy())

[0 0 0 0 0 1 1 1]


C:\Users\eerieKNIGHT\AppData\Local\Temp\ipykernel_34132\2136763999.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [53]:
class SiameseNetwork(nn.Module):# A simple implementation of siamese network, ResNet50 is used
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(3, 64, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Dropout2d(p=.2),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Dropout2d(p=.2),

            nn.ReflectionPad2d(1),
            nn.Conv2d(64, 32, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.Dropout2d(p=.2),
        )
        self.fc1 = nn.Linear(2*32*100*100, 500)
        self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(500, 2)


    def forward(self, input1, input2)
        output1 = self.cnn1(input1)
        output1 = output1.view(output1.size()[0], -1)#make it suitable for fc layer.
        output2 = self.cnn1(input2)
        output2 = output2.view(output2.size()[0], -1)
        
        output = torch.cat((output1, output2),1)
        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))
        output = self.fc3(output)
        return output

In [54]:
net = SiameseNetwork().cuda()
criterion = nn.CrossEntropyLoss() # use a Classification Cross-Entropy loss
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

counter = []
loss_history = [] 
iteration_number= 0

for epoch in range(0,NUMBER_EPOCHS):
    print("Epoch：", epoch, " start.")
    for i, data in enumerate(trainloader,0):
        img0, img1 , labels = data 
        img0, img1 , labels = img0.cuda(), img1.cuda() , labels.cuda()#move to GPU
        optimizer.zero_grad()#clear the calculated grad in previous batch
        outputs = net(img0,img1)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        if i %10 == 0 :#show changes of loss value after each 10 batches
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss.item())
    
    #test the network after finish each epoch
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for data in valloader:
            img0, img1 , labels = data
            img0, img1 , labels = img0.cuda(), img1.cuda() , labels.cuda()
            outputs = net(img0,img1)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            
    print('Accuracy of the network on the', total_val, 'val pairs in',val_famillies, ': %d %%' % (100 * correct_val / total_val))
    show_plot(counter,loss_history)

Epoch： 0  start.
Accuracy of the network on the 296 val pairs in F09 : 56 %
Epoch： 1  start.


C:\Users\eerieKNIGHT\AppData\Local\Temp\ipykernel_34132\2136763999.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Accuracy of the network on the 296 val pairs in F09 : 59 %
Epoch： 2  start.
Accuracy of the network on the 296 val pairs in F09 : 54 %
Epoch： 3  start.
Accuracy of the network on the 296 val pairs in F09 : 63 %
Epoch： 4  start.
Accuracy of the network on the 296 val pairs in F09 : 57 %
Epoch： 5  start.
Accuracy of the network on the 296 val pairs in F09 : 61 %
Epoch： 6  start.
Accuracy of the network on the 296 val pairs in F09 : 57 %
Epoch： 7  start.
Accuracy of the network on the 296 val pairs in F09 : 57 %
Epoch： 8  start.
Accuracy of the network on the 296 val pairs in F09 : 61 %
Epoch： 9  start.
Accuracy of the network on the 296 val pairs in F09 : 63 %
Epoch： 10  start.
Accuracy of the network on the 296 val pairs in F09 : 64 %
Epoch： 11  start.
Accuracy of the network on the 296 val pairs in F09 : 55 %
Epoch： 12  start.
Accuracy of the network on the 296 val pairs in F09 : 57 %
Epoch： 13  start.
Accuracy of the network on the 296 val pairs in F09 : 62 %
Epoch： 14  start.
Accurac

In [55]:
class testDataset(Dataset):
    
    def __init__(self,transform=None):
        self.test_df = pd.read_csv(os.getcwd() + '/input/sample_submission.csv')
        self.transform = transform
        
    def __getitem__(self,index):
        #data in submission.csv:
        #       img_pair               is_related
        #face05508.jpg-face01210.jpg       0
        #face05820.jpg-face03938.jpg       0
        
        img0_path = self.test_df.iloc[index].img_pair.split("-")[0]
        img1_path = self.test_df.iloc[index].img_pair.split("-")[1]
        
        img0 = Image.open(os.getcwd() + '/input/test/'+img0_path)
        img1 = Image.open(os.getcwd() + '/input/test/'+img1_path)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1
    
    def __len__(self):
        return len(self.test_df)

In [56]:
testset = testDataset(transform=transforms.Compose([transforms.Resize((IMG_SIZE,IMG_SIZE)),
                                                                      transforms.ToTensor()
                                                                      ]))
testloader = DataLoader(testset,
                        shuffle=False,
                        batch_size=1)

In [57]:
test_df = pd.read_csv(os.getcwd() + '/input/sample_submission.csv')
predictions=[]
with torch.no_grad():
    for data in testloader:
        img0, img1 = data
        img0, img1 = img0.cuda(), img1.cuda()
        outputs = net(img0,img1)
        _, predicted = torch.max(outputs, 1)
        predictions = np.concatenate((predictions,predicted.cpu().numpy()),0)
        
test_df['is_related'] = predictions
test_df.to_csv("submission.csv", index=False)
test_df.head(50)

,img_pair,is_related
0,face05508.jpg-face01210.jpg,1.0
1,face05750.jpg-face00898.jpg,1.0
2,face05820.jpg-face03938.jpg,1.0
3,face02104.jpg-face01172.jpg,0.0
4,face02428.jpg-face05611.jpg,0.0
5,face01219.jpg-face00274.jpg,0.0
6,face04262.jpg-face00555.jpg,0.0
7,face03697.jpg-face01892.jpg,0.0
8,face03524.jpg-face00319.jpg,1.0
9,face03410.jpg-face05368.jpg,0.0
